In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("Biosample_all.tsv.gz", sep="\t")

In [3]:
# replace nan with "", this is faster than calling df = df.fillna("")
df = df.applymap(lambda x: "" if pd.isna(x) else x)

In [4]:
df[0:5]

,BIOSAMPLE_ID,BIOSAMPLE_NAME,ADD_DATE,MOD_DATE,MOD_BY,DESCRIPTION,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,...,DEPTH2,ALTITUDE,ALTITUDE2,SOLUBLE_IRON_MICROMOL,BICARBONATE_MILLIMOL,H2S_MILLIMOL,H2S_PRESENT,IRRADIANCE,OXYGEN_PRESENCE,METHANE_CONC_MILLIMOL
0,173480,Wine grape associated microbial communities fr...,09-DEC-17 11.15.52.119000000 PM,05-JAN-18 12.58.38.000000000 PM,102151,Wine grape associated microbial communities fr...,Host-associated,Plants,Phyllosphere,Caulosphere,...,,,,,,,,,,
1,173774,Fermented milk associated microbial communitie...,09-DEC-17 11.36.30.369000000 PM,05-JAN-18 12.54.38.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,
2,173775,Fermented milk associated microbial communitie...,09-DEC-17 11.36.34.360000000 PM,05-JAN-18 12.54.43.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,
3,173776,Fermented milk associated microbial communitie...,09-DEC-17 11.36.38.315000000 PM,05-JAN-18 12.54.43.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,
4,173784,Fermented milk associated microbial communitie...,09-DEC-17 11.37.10.939000000 PM,05-JAN-18 12.54.43.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,


In [32]:
list(df.columns)

['BIOSAMPLE_ID',
 'BIOSAMPLE_NAME',
 'ADD_DATE',
 'MOD_DATE',
 'MOD_BY',
 'DESCRIPTION',
 'ECOSYSTEM',
 'ECOSYSTEM_CATEGORY',
 'ECOSYSTEM_TYPE',
 'ECOSYSTEM_SUBTYPE',
 'SPECIFIC_ECOSYSTEM',
 'SAMPLE_COLLECTION_SITE',
 'ISOLATION_PUBLICATION_ID',
 'SAMPLE_ISOLATION_COMMENTS',
 'SAMPLING_STRATEGY',
 'REPLICATE_NUMBER',
 'SAMPLE_VOLUME',
 'SAMPLE_BIOMASS',
 'SAMPLE_CONTACT_NAME',
 'SAMPLE_CONTACT_EMAIL',
 'GEOGRAPHIC_LOCATION',
 'XLATITUDE',
 'XLONGITUDE',
 'LAT_LONG_INFERRED',
 'XALTITUDE',
 'XDEPTH',
 'TEMPERATURE_RANGE',
 'TEMPERATURE_EXACT',
 'SALINITY',
 'PRESSURE',
 'PH',
 'HABITAT',
 'HOST_NAME',
 'HOST_TAXONOMY_ID',
 'HOST_GENDER',
 'HOST_RACE',
 'HOST_AGE',
 'HOST_HEALTH_CONDITION',
 'PATIENT_VISIT_NUMBER',
 'HOST_MEDICATION',
 'MRN',
 'HOST_BODY_SITE',
 'HOST_BODY_SUBSITE',
 'HOST_BODY_PRODUCT',
 'HOST_SPECIFICITY',
 'HOST_COMMENTS',
 'ACTIVE',
 'PROJECT_OID',
 'SAMPLE_OID',
 'GOLD_ID',
 'IS_PUBLIC',
 'SPECIMEN',
 'SUBMIT_BIOSAMPLE_NAME',
 'NCBI_TAXONOMY_ID',
 'COMMUNITY',
 'LOC

In [9]:
elevels = ['ECOSYSTEM', 'ECOSYSTEM_CATEGORY', 'ECOSYSTEM_TYPE', 'ECOSYSTEM_SUBTYPE', 'SPECIFIC_ECOSYSTEM']
df[elevels][0:10]

,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,SPECIFIC_ECOSYSTEM
0,Host-associated,Plants,Phyllosphere,Caulosphere,
1,Engineered,Food production,Dairy products,,
2,Engineered,Food production,Dairy products,,
3,Engineered,Food production,Dairy products,,
4,Engineered,Food production,Dairy products,,
5,Engineered,Food production,Dairy products,,
6,Engineered,Food production,Dairy products,,
7,Environmental,Aquatic,Freshwater,River,
8,Environmental,Aquatic,Freshwater,River,
9,Environmental,Aquatic,Freshwater,Lake,Sediment


In [5]:
## Add paths

def mkpath(row):
    s = False
    toks = []
    for l in elevels[::-1]:
        v = row[l]
        if (v == "" or v == "Unclassified") and not s:
            continue
        else:
            toks.append(v)
    toks.reverse()
    #print(str(toks))
    return ":".join(toks)



In [6]:
# len(df['ECOSYSTEM_CATEGORY'].unique())
# df['ECOSYSTEM_CATEGORY'].unique()

In [10]:
df['path'] = df.apply(lambda r : mkpath(r), axis=1)

In [11]:
df['path'][0:10]

0    Host-associated:Plants:Phyllosphere:Caulosphere
1          Engineered:Food production:Dairy products
2          Engineered:Food production:Dairy products
3          Engineered:Food production:Dairy products
4          Engineered:Food production:Dairy products
5          Engineered:Food production:Dairy products
6          Engineered:Food production:Dairy products
7             Environmental:Aquatic:Freshwater:River
8             Environmental:Aquatic:Freshwater:River
9     Environmental:Aquatic:Freshwater:Lake:Sediment
Name: path, dtype: object

In [12]:
## num distinct paths
len(set(df['path']))

461

## Find primitives that exist at multiple levels


In [30]:
for c in elevels:
    sc = set(df[c])
    sc.discard("")
    sc.discard("Unclassified")
    for d in elevels:
        if c <= d: # BD: This seems to assume that order of the columns matters ... does it? Does c == d make more sense?
            continue
        sd = set(df[d])
        ixn = sc.intersection(sd)
        if len(ixn) > 0:
            print(' * {} /\ {} = {}'.format(c,d,ixn))
        

 * ECOSYSTEM_TYPE /\ ECOSYSTEM_CATEGORY = {'Fungi', 'Echinodermata', 'Cnidaria', 'Wastewater'}
 * ECOSYSTEM_TYPE /\ ECOSYSTEM_SUBTYPE = {'Soil', 'Nodule', 'Sediment', 'Wastewater', 'Volcanic', 'Wood'}
 * ECOSYSTEM_SUBTYPE /\ ECOSYSTEM_CATEGORY = {'Wastewater', 'Endosymbionts'}
 * SPECIFIC_ECOSYSTEM /\ ECOSYSTEM_CATEGORY = {'Bioreactor'}
 * SPECIFIC_ECOSYSTEM /\ ECOSYSTEM_TYPE = {'Sediment', 'Tissue'}
 * SPECIFIC_ECOSYSTEM /\ ECOSYSTEM_SUBTYPE = {'Mine', 'Sediment', 'Microbialites', 'Fecal', 'Alkaline'}


## Translate TSV into a graph

 * Each node is a primitive
 * parents = primitives in the level above
 * children = primitives in the level below

In [14]:
import networkx as nx

In [15]:
## Turn dataframe into a graph
node2paths = {}
def df2g(df):
    G = nx.DiGraph()
    for ix, row in df.iterrows():
        path = ""
        for i in range(1,len(elevels)):
            p = row[elevels[i-1]]
            c = row[elevels[i]]
            path += ">" + p
            if c != "" and c != "Unclassified":
                G.add_edge(p,c)
                if c not in node2paths:
                    node2paths[c] = set()
                node2paths[c].add(path)
    return G

G = df2g(df)          

In [18]:
list(G.predecessors('Solid waste'))

['Engineered']

In [19]:
list(G.successors('Solid waste'))

['Solid animal waste', 'Landfill', 'Composting', 'Swine wastewater']

In [20]:
## Total primitives
len(G.nodes())

369

## Primitives that exist in more than one place

In [24]:
# for n in G.nodes_iter():
for n in G.nodes():
    ps = [p for p in G.predecessors(n) if p != "Unclassified"]
    if len(ps) > 1:
        print(" * {} parents: {}".format(n, ps))
        for path in node2paths[n]:
            print("     * PATH: {}".format(path))

 * Sediment parents: ['Lake', 'Freshwater', 'Hydrothermal vents', 'Aquatic', 'Coastal', 'Oceanic', 'Sediment', 'Hot (42-90C)', 'Thermal springs', 'Wetlands', 'Pond', 'Intertidal zone', 'Geologic', 'Hypersaline', 'Lentic', 'Neritic zone', 'Saline', 'Marine', 'Lotic', 'Alkaline', 'Cold seeps', 'Subtidal zone', 'Floodplain']
     * PATH: >Environmental>Aquatic>Non-marine Saline and Alkaline>Hypersaline
     * PATH: >Environmental>Aquatic>Freshwater>Pond
     * PATH: >Environmental>Aquatic>Marine>Hydrothermal vents
     * PATH: >Environmental>Aquatic>Marine>Intertidal zone
     * PATH: >Environmental>Aquatic>Marine>Neritic zone
     * PATH: >Environmental>Aquatic>Freshwater>Wetlands
     * PATH: >Environmental>Aquatic>Thermal springs>Hot (42-90C)
     * PATH: >Environmental>Aquatic>Marine>Subtidal zone
     * PATH: >Environmental>Aquatic>Freshwater>Lake
     * PATH: >Environmental>Aquatic>Marine>Oceanic
     * PATH: >Environmental>Aquatic>Freshwater>Lotic
     * PATH: >Environmental>Aquati

In [29]:
# import networkx as nx

# for i in range(0,len(elevels)):
#     li = elevels[i]
#     for ci in set(df[ci]):
        
#         for j in range(i,elevels):
#             lj = elevels[j]
        

## Depth

In [27]:
depth_df = df[df.DEPTH != ""]
depth_df

,BIOSAMPLE_ID,BIOSAMPLE_NAME,ADD_DATE,MOD_DATE,MOD_BY,DESCRIPTION,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,...,ALTITUDE,ALTITUDE2,SOLUBLE_IRON_MICROMOL,BICARBONATE_MILLIMOL,H2S_MILLIMOL,H2S_PRESENT,IRRADIANCE,OXYGEN_PRESENCE,METHANE_CONC_MILLIMOL,path
7,173867,Freshwater microbial communities from Amazon R...,09-DEC-17 11.43.44.472000000 PM,05-JAN-18 12.54.24.000000000 PM,102151,Freshwater microbial communities from Amazon R...,Environmental,Aquatic,Freshwater,River,...,,,,,,,,,,Environmental:Aquatic:Freshwater:River
8,173872,Freshwater microbial communities from Amazon R...,09-DEC-17 11.44.06.993000000 PM,05-JAN-18 12.54.24.000000000 PM,102151,Freshwater microbial communities from Amazon R...,Environmental,Aquatic,Freshwater,River,...,,,,,,,,,,Environmental:Aquatic:Freshwater:River
9,173903,Lake sediment microbial communtites from St. P...,09-DEC-17 11.47.38.822000000 PM,20-APR-18 06.00.14.000000000 PM,101232,Lake sediment microbial communtites from St. P...,Environmental,Aquatic,Freshwater,Lake,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Lake:Sediment
61,156244,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.31.216000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
62,156255,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.36.771000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
63,156276,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.47.524000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
64,156280,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.49.664000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
65,156281,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.50.190000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
66,156288,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.53.934000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
67,156295,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.57.631000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater


In [28]:
set(depth_df.ECOSYSTEM_CATEGORY)

{'Air',
 'Algae',
 'Annelida',
 'Aquatic',
 'Arthropoda',
 'Bioreactor',
 'Bioremediation',
 'Biotransformation',
 'Built environment',
 'Cnidaria',
 'Food production',
 'Fungi',
 'Human',
 'Insecta',
 'Invertebrates',
 'Lab enrichment',
 'Mammals',
 'Microbial',
 'Modeled',
 'Mollusca',
 'Plants',
 'Porifera',
 'Solid waste',
 'Terrestrial',
 'Tunicates',
 'Unclassified',
 'Wastewater'}